In [ ]:
# Creo il DataFrame
import pandas as pd
from MJP.majority_portfolio_utilities_TV import * 
import os

FILENAME = 'monthly_data.csv'
factors_df=pd.DataFrame({
    'factors': ['ag','beta','bm','cumret','dolvol6','gp','ill6','ns','size','volatility','acc'],
    'signs': [  -1.,  1.,   1.,    1.,     -1.,     1.,    1.,  -1.,  -1.,   -1.,   -1.],
    'wsigns':[  -1.,  1.,   1.,    1.,     -1.,     1.,    1.,  -1.,  -1.,   -1.,   -1.]
})

DATA_INIZIO = '2000-01-01'
DATA_FINE = '2023-12-31'

df = create_df(FILENAME, factors_df['factors'].tolist(), DATA_INIZIO, DATA_FINE) # funzione sua 
print(f"DataFrame caricato: {df.shape[0]:,} righe, {df.shape[1]} colonne")
df.head()

In [30]:
# Normalizzo il df tramite quantile-normalization
from sklearn.preprocessing import QuantileTransformer

col_to_norm = [col for col in df.columns if col not in ['PERMNO', 'date']]
df = df.dropna(subset=col_to_norm)


scaler = QuantileTransformer(output_distribution='uniform', random_state=0)
df_norm = df.copy()
df_norm[col_to_norm] = scaler.fit_transform(df[col_to_norm])

print('df normalizzato! ')
df_norm.head()

df normalizzato! 


,PERMNO,date,RET,ns,me,size,me_lag,ag,gp,acc,bm,RF,Mkt_RF,RET_RF,cumret,dolvol6,ill6,volatility,beta
0,10001,2000-01-31,0.329939,0.556602,0.085656,0.085657,0.089232,0.420614,0.189137,0.09881,0.765328,0.889890,0.138639,0.321699,0.403591,0.050255,0.869804,0.038279,0.092667
1,10001,2000-02-29,0.550551,0.556602,0.087297,0.087298,0.085505,0.420614,0.189137,0.09881,0.759953,0.921922,0.647648,0.537564,0.452151,0.045478,0.878527,0.016319,0.092918
2,10001,2000-03-31,0.426884,0.625028,0.084890,0.084891,0.086477,0.420614,0.189137,0.09881,0.768083,0.955455,0.857357,0.414703,0.454727,0.045289,0.866558,0.016411,0.095125
3,10001,2000-04-30,0.534196,0.625028,0.085822,0.085823,0.084971,0.420614,0.189137,0.09881,0.764979,0.948448,0.089089,0.521539,0.423346,0.043078,0.827210,0.016173,0.089287
4,10001,2000-05-31,0.400250,0.625028,0.084226,0.084227,0.085603,0.420614,0.189137,0.09881,0.772073,0.974474,0.144144,0.386798,0.457693,0.038532,0.852031,0.016422,0.093883


In [ ]:
# Creo portafoglio MJ
remove_outliers = False
inf, sup = None, None
MJ_configuration={
    'K' : 12, # holding_periods in months
    'lag' : 5, # 5 for june reallocation
    'factors' : factors_df['factors'].to_list(),
    'num_port' : 10, # number of portfolios
    'num_cat' : 6, # number of categorical variables to use per factor in mj #'num_cat': 6
    'weighting': False,
    'verbose': True,
    'n_jobs':-1,
    'mj_window':1,
    'method' : 'majority', # mean_rank, majority, 75q, 90q, lex, dlex
    'rolling_method':'profile',  ###'rank','vote','profile'
    'treat_na_mj':'median',
    'remove_outliers':remove_outliers,
    'inclusive':True,
    'fix_signs':True, # True: tieni fisso il numero di votanti ed il loro segno
    'all_voters_not_nan_on_reallocation':True,
    #### PARAMETRI UTILIZZATI SOLO SE fix_signs=False ####
    'min_voters':5,    
    'voting_window':6,
    'sign_voting_window':12,
    'p_threshold' : 0.1, 
    'delta_utility' : 0,
    'eliminations': 1,
    'players_batch_size': 5,
    'small': True
}
if remove_outliers:
    MJ_configuration['outliers']=[inf,sup]

if MJ_configuration['weighting']:
    MJ_configuration['default_voters']=factors_df['factors'].to_list()
    MJ_configuration['default_signs']=factors_df['signs'].to_list()
    compute='mj'
elif not MJ_configuration['weighting']:
    MJ_configuration['default_voters']=factors_df['factors'].to_list()
    MJ_configuration['default_signs']=factors_df['wsigns'].to_list()
    compute='wmj'

portfolios_MJ, _, _, = compute_MJ_portfolio_strategy(df_norm, MJ_configuration)
# '2020-06-30': [list of tickers]   '2021-06-30': [list of tickers]

# Tiro fuori il migliore
k = 'wmj' if MJ_configuration['weighting'] else 'mj' # wmj se uso i pesi, altrimenti mj
df_MJ = portfolios_MJ['mj']
rendimenti_medi = df_MJ.mean() # Calcolo i rendimenti medi di ciascun portf
best_MJ = rendimenti_medi.idxmax()
best_rendimento_MJ = rendimenti_medi.max()
print(f"Il portafoglio migliore per MJ e' {best_MJ} con rendimento medio {best_rendimento_MJ}")
df_best_MJ = df_MJ[[best_MJ]] # Estraggo solo la colonna giusta

Il portafoglio migliore per MJ e' port4 con rendimento medio 0.5117473139970752


In [ ]:
# Creare i portafogli con gli altri metodi
# Punto 1 --> TOPSIS